This notebook is used to generate the 3D poses of sequences using yolo and save them in a .json file.

In [1]:
from egoexo_dataloader_2 import EgoExoDataset
from ultralytics import YOLO
import os

In [2]:
dataset_path = '/media/thibault/T5 EVO/Datasets/Ego4D/'
train_dataset = EgoExoDataset(dataset_path, os.path.join(dataset_path, 'takes.json'), split = "train", viewpoint = "exo", skill=True, get_frames=True, get_pose=False, get_hands_pose=False, frame_rate=3, transform=None)
val_dataset = EgoExoDataset(dataset_path, os.path.join(dataset_path, 'takes.json'), split = "val",viewpoint = "exo",skill=True, get_frames=True, get_pose=False, get_hands_pose=False, frame_rate=3, transform=None)

In [3]:
data = train_dataset.__getitem__(10)


KeyboardInterrupt: 

In [4]:
paths = [path for path in train_dataset.samples[0]["exo"]]
paths.append(train_dataset.samples[0]["ego"])
paths

['/media/thibault/T5 EVO/Datasets/Ego4D/takes/georgiatech_cooking_01_01_4/frame_aligned_videos/downscaled/448/aria04.mp4',
 '/media/thibault/T5 EVO/Datasets/Ego4D/takes/georgiatech_cooking_01_01_4/frame_aligned_videos/downscaled/448/cam01.mp4',
 '/media/thibault/T5 EVO/Datasets/Ego4D/takes/georgiatech_cooking_01_01_4/frame_aligned_videos/downscaled/448/cam02.mp4',
 '/media/thibault/T5 EVO/Datasets/Ego4D/takes/georgiatech_cooking_01_01_4/frame_aligned_videos/downscaled/448/cam03.mp4',
 '/media/thibault/T5 EVO/Datasets/Ego4D/takes/georgiatech_cooking_01_01_4/frame_aligned_videos/downscaled/448/cam04.mp4',
 '/media/thibault/T5 EVO/Datasets/Ego4D/takes/georgiatech_cooking_01_01_4/frame_aligned_videos/downscaled/448/aria04_214-1.mp4']

In [ ]:
import cv2
import numpy as np
from typing import Callable, Generator, List

def frame_windows(video_path: str,
                  window_size: int
                 ) -> Generator[List[np.ndarray], None, None]:
    """
    Lazily read `window_size` frames at a time from the video.
    Yields lists of frames (last batch may be smaller).
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video {video_path}")
    
    frames: List[np.ndarray] = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        if len(frames) == window_size:
            yield frames
            frames = []
    if frames:
        yield frames
    cap.release()

def extract_skeleton(video_path: str,
                     window_size: int = 32,
                     predict: Callable[[List[np.ndarray]], np.ndarray] = None
                    ) -> np.ndarray:
    """
    Process a video in non-overlapping windows to extract per-frame skeletons
    without ever loading the whole video into RAM at once.

    Args:
        video_path: Path to the input video file.
        window_size: Number of frames per chunk.
        predict: Function(frames_list) → np.ndarray of shape
                 (num_frames, num_keypoints, dims).

    Returns:
        skeleton_sequence: np.ndarray of shape
            (total_frames, num_keypoints, dims).
    """
    if predict is None:
        raise ValueError("You must pass in a `predict` function.")
    
    skeleton_chunks = []
    total = 0
    for frames in frame_windows(video_path, window_size):
        # frames is at most window_size in length
        sk = predict(frames)  # e.g. (len(frames), K, D)
        skeleton_chunks.append(sk)
        total += sk.shape[0]
    
    # Concatenate along temporal axis:
    skeleton_sequence = np.concatenate(skeleton_chunks, axis=0)
    assert skeleton_sequence.shape[0] == total
    return skeleton_sequence


torch.Size([16, 3, 448, 796])


In [ ]:
skeleton_sequence = extract_skeleton(paths[1], )